ASSIGNMENT 2 - GROUP 11

PLEASE NOTE: The features we decided to extract using spacy distance take too long to run (approximately 5 hours). This is -- we believe -- due to the fact that the en_core_web_lg pipeline is taking a long time, as it applies many pre-processing steps before extracting the feature. See here: https://spacy.io/models/en#en_core_web_lg. As such, we have submitted the various feature files we exported in CODE BLOCK 14, along with our report. You can import them directly in the 'REGRESSION' section (CODE BLOCK 15), and run the notebook from there to inspect the regressions.

In [4]:
# CODE BLOCK 1: IMPORTS
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import SGDRegressor, ElasticNet, Ridge
from sklearn.svm import SVR, LinearSVR
from nltk.stem.snowball import SnowballStemmer
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
# !python3 -m spacy download en_core_web_lg # NOTE: YOU MIGHT NEED TO UNCOMMENT THIS LINE AND DOWNLOAD THIS MODEL
nlp = spacy.load("en_core_web_lg")
import en_core_web_lg
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import os
import re
import math
pd.set_option('display.max_colwidth', None)
import csv
import matplotlib as plt

[nltk_data] Downloading package punkt to /Users/elena/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
# CODE BLOCK 2: IMPORTING DATA FOR PRE-PROCESSING

df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
df_test = pd.read_csv('test.csv', encoding = 'latin-1')
df_attr = pd.read_csv('attributes.csv', encoding = 'ISO-8859-1')
df_pro_desc = pd.read_csv('product_descriptions.csv', encoding = 'latin-1')

len_training = df_train.shape[0]
df_train = df_train[:len_training]
df_test = df_test[len_training:]
pro_desc = df_pro_desc[:len_training]

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, pro_desc, how='left', on='product_uid')


In [25]:
# CODE BLOCK 3: COMPUTING COUNT FEATURES (PART OF BASELINE METHOD)

stemmer = SnowballStemmer('english')

def str_stemmer(s):
	if type(s) == str:
		return " ".join([stemmer.stem(word) for word in s.lower().split()])
	else:
		return


# Changed this function to handle NaN values
def str_common_word(str1, str2):
	if type(str1) == str and type(str2) == str:
		return sum(int(str2.find(word)>=0) for word in str1.split())
	else:
		return


# STEM THE ATTRIBUTES WE WANT TO TURN INTO COUNT FEATURES
df_all['search_term_stem'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title_stem'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description_stem'] = df_all['product_description'].map(lambda x:str_stemmer(x))

# CALCULATE THE LENGTH OF THE SEARCH QUERY
df_all['len_of_query'] = df_all['search_term_stem'].map(lambda x:len(x.split())).astype(np.int64)

# COMBINE ALL STEMMED ATTRIBUTES INTO ONE COLUMN
df_all['product_info_stemmed'] = df_all['search_term_stem']+"\t"+df_all['product_title_stem']+"\t"+df_all['product_description_stem']

# COMBINE ALL UNSTEMMED ATTRIBUTES INTO ONE COLUMN
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

# COUNT THE COMMON WORDS BETWEEN THE SEARCH TERM AND EACH ATTRIBUTE (STEMMED)
df_all['word_in_title_stemmed'] = df_all['product_info_stemmed'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
df_all['word_in_description_stemmed'] = df_all['product_info_stemmed'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)

# COUNT THE COMMON WORDS BETWEEN THE SEARCH TERM AND EACH ATTRIBUTE (UNSTEMMED)
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]) if type(x) == str else 0)
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x) == str else 0)


In [26]:
# CODE BLOCK 4: COMPUTING NEW COUNT FEATURES

## FROM HERE, NEW FEATURE EXTRACTION NOT IN THE YAO JEN CHANG CODE

# ADDING ATTRIBUTES - COUNT OF PRODUCT ATTRIBUTES IN THE SEARCH TERM
attributes = df_attr.copy()
df_attr['all_values'] = df_attr[['product_uid', 'value']].groupby(['product_uid'])['value'].transform(lambda x : ','.join((word_tokenize(stemmer.stem(str(x))))))
df_all_w_attr = df_all.join(df_attr, how = 'left', lsuffix = '_left', rsuffix='_right')
df_all_w_attr['search_term_and_all_attributes'] = df_all_w_attr['search_term']+"\t"+df_all_w_attr['name'].transform(lambda x : ','.join(word_tokenize(str(x))))+"\t"+df_all_w_attr['all_values']
df_all_w_attr['word_in_attributes'] = df_all_w_attr['search_term_and_all_attributes'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]) if type(x)==str else 0)


In [27]:
# CODE BLOCK 5: ATTRIBUTE - VALUE PAIRS PRE-PROCESSING
attributes = attributes.astype({"name":str})
grouped_attr = attributes.groupby(["product_uid", "name"], group_keys=False).agg(lambda x: x)
grouped_attr = grouped_attr.reset_index()
grouped_attr.columns = ["product_uid", "attr_name", "attr_value"]

In [28]:
# CODE BLOCK 6: REPORT QUERY

# This query to answer the question in the introduction regarding the most common brands
most_used_brands = grouped_attr[grouped_attr["attr_name"] == "MFG Brand Name"].groupby(["attr_value"]).count()

most_used_brands

,product_uid,attr_name
attr_value,,
.N/A,19,19
1-2-3Mortar,1,1
30 Seconds,1,1
3M,193,193
3M Tekk Protection,3,3
...,...,...
stufurhome,31,31
tattletale,2,2
the great outdoors by Minka Lavery,31,31


In [29]:
# CODE BLOCK 7: EXTRACT MOST COMMON ATTRIBUTES

types_of_attributes = grouped_attr["attr_name"].value_counts()
types_of_attributes = types_of_attributes.to_frame()
types_of_attributes = types_of_attributes.reset_index()
bullets = types_of_attributes["attr_name"].str.contains("Bullet")
types_of_attributes = types_of_attributes[~bullets]

most_used_attributes = types_of_attributes.iloc[0:20]
most_used_attribute_list = []

for i in most_used_attributes["attr_name"]:
    most_used_attribute_list.append(i)

grouped_attr = grouped_attr[grouped_attr["attr_name"].isin(most_used_attribute_list)]
attributes_as_cols = grouped_attr.pivot(index = "product_uid", columns="attr_name", values="attr_value")


In [30]:
# CODE BLOCK 8: PROCESS MEASUREMENT FEATURES

def cleanup_text_in_df(text, dataframe, column):
    dataframe[column] = dataframe[column].str.replace(text, '')
    return dataframe


columns_to_clean = ["Assembled Height (in.)", "Assembled Width (in.)", "Product Depth (in.)", "Product Height (in.)", "Product Length (in.)", "Product Width (in.)"]
attributes_as_cols = cleanup_text_in_df("in", attributes_as_cols, "Assembled Depth (in.)")
for column in columns_to_clean:
    attributes_as_cols = cleanup_text_in_df("in", attributes_as_cols, column)


def make_one_measurement(depth_col, width_col, length_col):
    def check_nan(col_name):
        nan = type(float('nan'))
        if type(col_name) != nan:
            return True
    measurements = []
    if check_nan(depth_col):
        depth = depth_col
        measurements.append(depth)
    if check_nan(width_col):
        width = width_col
        measurements.append(width)
    if check_nan(length_col):
        length = length_col
        measurements.append(length)
    measurement_str = (" x ").join(measurements)
    return measurement_str


attributes_as_cols["Assembled Measurement (DxWxH)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Assembled Depth (in.)"], x["Assembled Height (in.)"], x["Assembled Width (in.)"]), axis = 1)
attributes_as_cols["Product Measurement (LxWxD)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Product Length (in.)"], x["Product Width (in.)"], x["Assembled Depth (in.)"]), axis = 1)
attributes_as_cols["Product Measurement (DxWxH)"] = attributes_as_cols.apply(lambda x: make_one_measurement(x["Product Length (in.)"], x["Product Width (in.)"], x["Assembled Height (in.)"]), axis = 1)
attributes_as_cols.to_csv("attributes_as_cols.csv")

In [31]:
# CODE BLOCK 9: DATA EXPLORATION TO DETERMINE HOW COMMON TITLE + MEASUREMENT IS IN THE SEARCH TERMS

def recognise_measurements(dataframe):
    measurements_in_term = {}
    term = None
    regex_1 = re.compile('\d+x\d+')
    regex_2 = re.compile('\d+\*\d+')
    for i in dataframe.itertuples():
        term_1 = regex_1.match(i[4])
        term_2 = regex_2.match(i[4])
        if term_1:
            measurements_in_term[i[2]]=term_1.group(0)
        if term_2:
            split_term = term_2.group(0).split("*")
            term_2 = split_term[0] + " x " + split_term[1]
            measurements_in_term[i[2]] = term_2
    return measurements_in_term


measurements = recognise_measurements(df_all)

In [32]:
# CODE BLOCK 10: MAKE PRODUCT TITLE + MEASUREMENTS FEATURE

measurements_in_product = attributes_as_cols[["Assembled Measurement (DxWxH)", "Product Measurement (LxWxD)", "Product Measurement (DxWxH)"]]
measurements_in_product.reset_index(inplace = True)
measurements_in_product.columns = ["product_uid", "Assembled Measurement (DxWxH)", "Product Measurement (LxWxD)", "Product Measurement (DxWxH)"]
df_all_w_attr = df_all_w_attr.join(measurements_in_product, how = 'left')

# Unstemmed
df_all_w_attr["title_and_assembled_measurement"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Assembled Measurement (DxWxH)"]
df_all_w_attr["title_and_measurement_LWD"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Product Measurement (LxWxD)"]
df_all_w_attr["title_and_measurement_DWH"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Product Measurement (DxWxH)"]

# Stemmed
df_all_w_attr["title_and_assembled_measurement_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Assembled Measurement (DxWxH)"]
df_all_w_attr["title_and_measurement_LWD_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Product Measurement (LxWxD)"]
df_all_w_attr["title_and_measurement_DWH_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Product Measurement (DxWxH)"]



In [34]:
# CODE BLOCK 11: MAKE PRODUCT TITLE + COLOUR FEATURE
colours = attributes_as_cols[["Color Family", "Color/Finish"]]
colours = colours.reset_index()
# Make Colour Stems
colours['Color Family Stem'] = colours['Color Family'].map(lambda x:str_stemmer(x))
colours['Color/Finish Stem'] = colours['Color/Finish'].map(lambda x:str_stemmer(x))

df_all_w_attr = df_all_w_attr.merge(colours, how = "left", on = "product_uid")
# Non-stemmed
df_all_w_attr["title_and_color"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Color/Finish"]
df_all_w_attr["title_and_color_fam"] = df_all_w_attr["product_title"] + " " + df_all_w_attr["Color Family"]
# Stemmed
df_all_w_attr["title_and_color_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Color/Finish Stem"]
df_all_w_attr["title_and_color_fam_stem"] = df_all_w_attr["product_title_stem"] + " " + df_all_w_attr["Color Family Stem"]




FEATURE EXTRACTION

In [35]:
# CODE BLOCK 12: SPACY
nlp = en_core_web_lg.load()

def spacy_similarity(doc_1_col, doc_2_col):
    doc_1 = nlp(doc_1_col)
    doc_2 = nlp(doc_2_col)
    return doc_1.similarity(doc_2)


# Spacy similarity score between just title and search term
df_all_w_attr["title_spacy_similarity"] = df_all_w_attr.apply(lambda x: spacy_similarity(x["product_title"], x["search_term"]), axis = 1)
df_all_w_attr

/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)


,id,product_uid_left,product_title,search_term,relevance,product_description,search_term_stem,product_title_stem,product_description_stem,len_of_query,...,title_and_measurement_DWH_stem,Color Family,Color/Finish,Color Family Stem,Color/Finish Stem,title_and_color,title_and_color_fam,title_and_color_stem,title_and_color_fam_stem,title_spacy_similarity
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl bracket,simpson strong-ti 12-gaug angl,"not onli do angl make joint stronger, they also provid more consistent, straight corners. simpson strong-ti offer a wide varieti of angl in various size and thick to handl light-duti job or project where a structur connect is needed. some can be bent (skewed) to match the project. for outdoor project or those where moistur is present, use our zmax zinc-coat connectors, which provid extra resist against corros (look for a ""z"" at the end of the model number).versatil connector for various 90 connect and home repair projectsstrong than angl nail or screw fasten alonehelp ensur joint are consist straight and strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad from 12-gaug steelgalvan for extra corros resistanceinstal with 10d common nail or #9 x 1-1/2 in. strong-driv sd screw",2,...,simpson strong-ti 12-gaug angl 3,NaN,NaN,None,None,NaN,NaN,NaN,NaN,0.283034
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",l bracket,simpson strong-ti 12-gaug angl,"not onli do angl make joint stronger, they also provid more consistent, straight corners. simpson strong-ti offer a wide varieti of angl in various size and thick to handl light-duti job or project where a structur connect is needed. some can be bent (skewed) to match the project. for outdoor project or those where moistur is present, use our zmax zinc-coat connectors, which provid extra resist against corros (look for a ""z"" at the end of the model number).versatil connector for various 90 connect and home repair projectsstrong than angl nail or screw fasten alonehelp ensur joint are consist straight and strongdimensions: 3 in. x 3 in. x 1-1/2 in.mad from 12-gaug steelgalvan for extra corros resistanceinstal with 10d common nail or #9 x 1-1/2 in. strong-driv sd screw",2,...,simpson strong-ti 12-gaug angl 7.76,Browns / Tans,Tugboat,brown / tan,tugboat,Simp

In [37]:
## CODE BLOCK 13: DISTANCE MEASURES WITH SPACY ##
# NOTE: This code block (which was split into smaller code blocks) originally took a few hours to run due to spacy being time consuming out-of-the-box. We suggest you skip to the import statements at the end to
# run the models

def apply_feature_score(newcolname, col1, col2, feature_scorer):
    df_all_w_attr[newcolname] = df_all_w_attr.apply(lambda x: feature_scorer(str(x[col1]), str(x[col2])), axis = 1)

# Spacy Similarity -- Unstemmed; used only for comparison
apply_feature_score("spacy_title_and_assembled_measurement", "title_and_assembled_measurement", "search_term", spacy_similarity)
apply_feature_score("spacy_title_and_measurement_LWD","title_and_measurement_LWD", "search_term", spacy_similarity)
apply_feature_score("spacy_title_and_measurement_DWH","title_and_measurement_DWH", "search_term", spacy_similarity)
apply_feature_score("spacy_title_and_color","title_and_color", "search_term", spacy_similarity)
apply_feature_score("spacy_title_and_color_fam","title_and_color_fam", "search_term", spacy_similarity)

# Spacy Similarity -- Stemmed
apply_feature_score("spacy_description_stemmed","product_description_stem", "search_term_stem", spacy_similarity)
apply_feature_score("spacy_title_stem_and_assembled_measurement", "title_and_assembled_measurement_stem", "search_term_stem", spacy_similarity)
apply_feature_score("spacy_title_and_measurement_LWD_stem", "title_and_measurement_LWD_stem", "search_term_stem", spacy_similarity)
apply_feature_score("spacy_title_and_measurement_DWH_stem", "title_and_measurement_DWH_stem", "search_term_stem", spacy_similarity)
apply_feature_score("spacy_title_and_color_stem","title_and_color_stem", "search_term_stem", spacy_similarity)
apply_feature_score("spacy_title_and_color_fam_stem","title_and_color_fam_stem", "search_term_stem", spacy_similarity)


/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)
/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)
/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)
/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)
/var/folders/wr/c8zkd12d081692wcbdb3w10c0000gn/T/ipykernel_67793/3449051160.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc_1.similarity(doc_2)


In [98]:
df_all_w_attr.columns

Index(['id', 'product_uid_left', 'product_title', 'search_term', 'relevance',
       'product_description', 'search_term_stem', 'product_title_stem',
       'product_description_stem', 'len_of_query', 'product_info_stemmed',
       'product_info', 'word_in_title_stemmed', 'word_in_description_stemmed',
       'word_in_title', 'word_in_description', 'product_uid_right', 'name',
       'value', 'all_values', 'search_term_and_all_attributes',
       'word_in_attributes', 'product_uid', 'Assembled Measurement (DxWxH)',
       'Product Measurement (LxWxD)', 'Product Measurement (DxWxH)',
       'title_and_assembled_measurement', 'title_and_measurement_LWD',
       'title_and_measurement_DWH', 'title_and_assembled_measurement_stem',
       'title_and_measurement_LWD_stem', 'title_and_measurement_DWH_stem',
       'Color Family', 'Color/Finish', 'Color Family Stem',
       'Color/Finish Stem', 'title_and_color', 'title_and_color_fam',
       'title_and_color_stem', 'title_and_color_fam_stem',

In [2]:
## CODE BLOCK 14: FINALISE & EXPORT DATASETS
unstemmed_all = df_all_w_attr[["id","relevance", "len_of_query", "word_in_title", "word_in_description", "word_in_attributes", "product_uid", "title_spacy_similarity", "spacy_title_and_assembled_measurement", "spacy_title_and_measurement_LWD", "spacy_title_and_measurement_DWH", "spacy_title_and_color", "spacy_title_and_color_fam", "spacy_description"]]
stemmed_all = df_all_w_attr[["id","relevance", "len_of_query", "word_in_title_stemmed", "word_in_description_stemmed", "word_in_attributes", "product_uid", "spacy_description_stemmed", "spacy_title_stem_and_assembled_measurement", "spacy_title_and_measurement_LWD_stem", "spacy_title_and_measurement_DWH_stem", "spacy_title_and_color_stem", "spacy_title_and_color_fam_stem"]]
counts_only = df_all_w_attr[["id","relevance", "len_of_query", "word_in_title", "word_in_description", "product_uid"]]
spacy_only = df_all_w_attr[["id","relevance", "product_uid", "spacy_description_stemmed", "spacy_title_stem_and_assembled_measurement", "spacy_title_and_measurement_LWD_stem", "spacy_title_and_measurement_DWH_stem", "spacy_title_and_color_stem", "spacy_title_and_color_fam_stem"]]


unstemmed_all.to_csv("unstemmed_attributes.csv")
stemmed_all.to_csv("stemmed_all.csv")
counts_only.to_csv("count_features_only.csv")
spacy_only.to_csv("spacy_only_features.csv")

NameError: name 'df_all_w_attr' is not defined

REGRESSION MODELS

Note: from this point on, we saved our pre-processed features to the csv (submitted along with our report and code), to save time when running. You can run the file from the cell below code to import the submitted .csv files instead of running the feature extraction cells from the beginning.

In [13]:
## CODE BLOCK 15: IMPORT PRE-PROCESSED DATASETS

df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
df_test = pd.read_csv('test.csv', encoding = 'latin-1')
df_attr = pd.read_csv('attributes.csv', encoding = 'ISO-8859-1')
df_pro_desc = pd.read_csv('product_descriptions.csv', encoding = 'latin-1')

len_training = df_train.shape[0]



unstemmed_all = pd.read_csv("unstemmed_attributes.csv")
stemmed_all = pd.read_csv("stemmed_all.csv")
counts_only = pd.read_csv("count_features_only.csv")
spacy_only = pd.read_csv("spacy_only_features.csv")


In [14]:
## CODE BLOCK 16: COMPARE STEMMED VS UNSTEMMED COUNTS (YAO-JEN CHANG ORIGINAL MODELS) -- ONLY FOR REPORT

stemmed_counts = stemmed_all[["id", "word_in_title_stemmed", "word_in_description_stemmed", "relevance", "len_of_query"]]
unstemmed_counts = unstemmed_all[["id","word_in_title", "word_in_description","relevance", "len_of_query"]]


def original_regression(feature_set):
  df_train = feature_set.iloc[:len_training]
  df_test = feature_set.iloc[len_training:]
  id_test = df_test['id']

  # Creating train-test split for assessing model performance
  y = df_train['relevance'].values
  X = df_train.drop(['id','relevance'],axis=1).values
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, shuffle = True)

  rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
  clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  rmse = mean_squared_error(y_test, y_pred, squared = False)
  return rmse

rmse_stemmed = original_regression(stemmed_counts)
rmse_unstemmed = original_regression(unstemmed_counts)

print(rmse_stemmed, rmse_unstemmed)

0.4908648805262448 0.5157199186738392


In [15]:
## CODE BLOCK 17: PREPARING REGRESSION MODELS

class Model():
    def __init__(self, label, estimator, random_parameters = None, grid_parameters = None ):
        if not grid_parameters and not random_parameters:
            raise Exception("Either random_parameters or grid_parameters have to be specified")
        
        self.label = label
        self.estimator = estimator
        self.random_parameters = random_parameters
        self.grid_parameters = grid_parameters


class Regressor():
    def __init__(self, dataset, sample_size = None):

        self.sample_size = sample_size

        # The processed training and testing sets, targets and labels combined
        self.train = dataset.iloc[:len_training]
        self.test = dataset.iloc[len_training:]
        id_test = df_test['id']
        self.pro_desc = None

        # Training and testing targets and labels.
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    def train_test_split(self, train_size:float= 0.8):
        if train_size != 1:
            # Creating train-test split for assessing model performance
            y = self.train['relevance'].values
            X = self.train.drop(['id','relevance'],axis=1).values
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, train_size = train_size, shuffle = True)

        else:
            # Using all training data for training, testing on actual test data
            self.y_train = self.train['relevance'].values
            self.X_train = self.train.drop(['id','relevance'],axis=1).values
            self.X_test = self.test.drop(['id','relevance'],axis=1).values

    def compare_models(self, models: dict, save_predictions: bool = False):
        all_rmse = {}
        for model in models.keys():

            clf = BaggingRegressor(models[model], n_estimators=45, max_samples=0.1, random_state=25)
            clf.fit(self.X_train, self.y_train)
            y_pred = clf.predict(self.X_test)

            rmse = mean_squared_error(self.y_test, y_pred, squared = False)
            
            print(f"{model}: {rmse}")
            all_rmse[model] = rmse


        rmse_df = pd.DataFrame.from_dict(all_rmse, orient = "index", columns = ['RMSE'])
        rmse_df.to_csv(f"{str(self.sample_size if self.sample_size != None else 'Full')}_{models.keys()}_rmse_comparison.csv")

        if save_predictions == True:
            pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

    def random_search(self, model: Model, n_iter = 20, random_state = 0):

        # Getting an estimate of hyperparams with Randomized Search
        model_random = RandomizedSearchCV(estimator = model.estimator, param_distributions = model.parameters,
                                          n_iter = n_iter,random_state = random_state,
                                          n_jobs= -1, scoring='neg_root_mean_squared_error')

        model_random.fit(self.X_train, self.y_train)
        print(f"model_random.best_score_: {model_random.best_score_}")
        print(f"model_random.best_params_: {model_random.best_params_}")

        random_summary = (model_random.best_params_ | {'RMSE': -(model_random.best_score_)})
        pd.DataFrame(random_summary, index = ['Best']).to_csv(f"{model.label}_random_summary.csv")

    def grid_search(self, model: Model, start = -0.2, stop = 0.2, num = 4, auto_params = True, verbose = 3):
        '''
        auto_params determines whether a narrow range should be constructed based on existing "optimal" parameters
        found through some other hyperparameter optimization like random search or another grid search
        auto params is false if no other hyperparameter optimization has been done.
        in that case, the full range of parameters are used.
        '''
        if not auto_params:
          params_grid = model.grid_parameters

        else:
          # Narrowing the search with Grid Search
          percent_range = np.linspace(start = start, stop = stop, num = num)

          random_params = pd.read_csv(f"{str(self.sample_size if self.sample_size != None else 'Full')}_{model.label}_random_summary.csv", index_col=0).drop('RMSE', axis = 'columns')

          # Building the parameters
          params_grid = dict()
          percent_range = np.linspace(start = -0.2, stop = 0.2, num = 3)
          for col in random_params:
              if type(random_params[col].iloc[0]) == np.int64 or type(random_params[col].iloc(0)) == np.float64:
                  if random_params[col].iloc[0] > 10:
                      param_range = [random_params[col].iloc[0] + int(x * random_params[col].iloc[0]) for x in percent_range]
                      params_grid.update({col:param_range})
                  else:
                      param_range = [random_params[col].iloc[0], random_params[col].iloc[0] + 1, random_params[col].iloc[0] + 2]
                      params_grid.update({col:param_range})
              else:
                  params_grid.update({col:[random_params[col].iloc[0]]})

        print("constructing search...")
        grid_search = GridSearchCV(estimator = model.estimator, param_grid=params_grid,n_jobs= -1, scoring='neg_root_mean_squared_error')
        print("fitting...")
        grid_search.fit(self.X_train,self.y_train)
        grid_summary = (grid_search.best_params_ | {'RMSE': -(grid_search.best_score_)})
        pd.DataFrame(grid_summary, index = ['Best']).to_csv(f"{str(self.sample_size if self.sample_size != None else 'Full')}_{model.label}_grid_summary.csv")


        feature_labels = np.array(self.train.columns.drop(["relevance","id"]))
        pd.DataFrame({"Coefficient":grid_search.best_estimator_.coef_}, index = feature_labels).to_csv(f"{str(self.sample_size if self.sample_size != None else 'Full')}_{model.label}_feature_coefs.csv")

    def compare_hyperparam_opt(self, model: Model):
        norm_rmse_df = pd.read_csv("rmse_comparison.csv")
        model_label = model.label
        grid_rmse = norm_rmse_df.model_label
        rand_rmse_df = pd.read_csv(f"{model.label}_random_summary.csv")
        rand_rmse = rand_rmse_df.RMSE
        grid_rmse_df = pd.read_csv(f"{model.label}_grid_summary.csv")
        grid_rmse = grid_rmse_df.RMSE


        hyperparam_scores = {'Normal': norm_rmse, 'RSCV': rand_rmse, 'GSCV': grid_rmse}
        print(f"hyperparam summary: {hyperparam_scores}")
        hyperparam_summary = pd.DataFrame.from_dict(hyperparam_scores, orient = "index", columns = ['RMSE'])
        hyperparam_summary.to_csv('hyperparam_summary.csv')

In [16]:
## CODE BLOCK 18: INITIALISING MODELS & DEFINING PARAMETERS


# Creating the rf model

rand_rf_bootstrap = [True, False]
rand_rf_n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000)]
rand_rf_max_features = [1.0, 'sqrt']
rand_rf_max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
rand_rf_max_depth.append(None)
rand_rf_min_samples_split = [2, 4, 6]
rand_rf_min_samples_leaf = [1, 2, 3]

# Creating the rf model

grid_rf_bootstrap = [True, False]
grid_rf_n_estimators = [int(x) for x in np.linspace(start = 5, stop = 105, num  = 11)]
grid_rf_max_features = [1.0, 'sqrt']
grid_rf_max_depth = [int(x) for x in np.linspace(5, 25, num = 5)]
grid_rf_max_depth.append(None)
grid_rf_min_samples_split = [2, 4, 6]
grid_rf_min_samples_leaf = [1, 2, 3]

rf = Model(
    label = 'random forest',
    estimator = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0),
    random_parameters = {
        'bootstrap': rand_rf_bootstrap,
        'n_estimators': rand_rf_n_estimators,
        'max_features': rand_rf_max_features,
        'max_depth': rand_rf_max_depth,
        'min_samples_split': rand_rf_min_samples_split,
        'min_samples_leaf': rand_rf_min_samples_leaf},
    grid_parameters = {
        'bootstrap': grid_rf_bootstrap,
        'n_estimators': grid_rf_n_estimators,
        'max_features': grid_rf_max_features,
        'max_depth': grid_rf_max_depth,
        'min_samples_split': grid_rf_min_samples_split,
        'min_samples_leaf': grid_rf_min_samples_leaf}

)

grid_en_parameters = dict()
grid_en_parameters['alpha'] = [0.01, 0.1, 1, 10, 100] # default is 1, 0 is another common choice. Adding ten for a log scale with 3 values.
grid_en_parameters['l1_ratio'] = [0, 0.25,0.5, 0.75, 1] # accepts values between 0 and 1, choosing 3 values with equal spacing in this range

en = Model(
    label = 'elastic net',
    estimator = ElasticNet(random_state=0),
    grid_parameters = grid_en_parameters
)


bagging_reg = Model(
    label = "bagging regressor",
    estimator = BaggingRegressor(estimator=RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0), n_estimators=45, max_samples=0.6, random_state=25),
    grid_parameters= {
        'bootstrap' : [True, False],
        'n_estimators' : rand_rf_n_estimators,
        'max_features' : [0.6, 0.8, 1.0]
    }
)

ridge = Model(
    label = 'ridge',
    estimator = Ridge(),
    grid_parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
)


In [17]:
## CODE BLOCK 19: INITIALISING REGRESSORS

unstemmed_reg = Regressor(dataset = unstemmed_all)
stemmed_reg = Regressor(dataset = stemmed_all)
counts_only_reg = Regressor(dataset = stemmed_counts)
spacy_only_reg = Regressor(dataset = spacy_only)

unstemmed_reg.train_test_split(train_size=0.8)
stemmed_reg.train_test_split(train_size=0.8)
counts_only_reg.train_test_split(train_size=0.8)
spacy_only_reg.train_test_split(train_size=0.8)


In [20]:
## CODE BLOCK 20: MODEL COMPARISONS

# Below, commented-out are some models that didn't give good results
# svrl = SVR(kernel='linear') # SVR (both linear and rbf) ran for 10+ minutes without completing
# svrr = SVR(kernel='rbf')
# sgd = SGDRegressor(random_state=0) # this gave a way-too large rmse. maybe have to use a scaler (see pipelines)

# These are the best models so far
models = {
    'Random Forest': RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0),
    'ElasticNet': ElasticNet(),
    'LinearSVR': LinearSVR(random_state=0),
    'Ridge': Ridge(random_state = 0),
    'BaggingRegressor': BaggingRegressor(estimator=RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0), n_estimators=45, max_samples=0.6, random_state=25)
    }


# COMPARING MODELS WITH DIFFERENT KINDS OF FEATURES -- UNCOMMENT TO TRY DIFFERENT FEATURE SETS
# unstemmed_reg.compare_models(models) # with only unstemmed features
# stemmed_reg.compare_models(models) # with only stemmed features
# counts_only_reg.compare_models(models) # with only count features
spacy_only_reg.compare_models(models) # with only semantic distance (spacy) features


Random Forest: 0.5246753857611899
ElasticNet: 0.5295861823055734


/Users/elena/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/elena/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/elena/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/elena/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/elena/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to 

LinearSVR: 0.5393891799194505
Ridge: 0.5283737016859381
BaggingRegressor: 0.5249022941118134


In [34]:
# CODE BLOCK 21: GRID SEARCH & FEATURE EVALUATION
# Note: here we kept the grid search and evaluation done on Ridge, but you can change it to any model defined in CODE BLOCK 18.

stemmed_all = pd.read_csv("stemmed_all.csv")
reg_data = stemmed_all.drop(["product_uid", "Unnamed: 0"],axis =1)
long_train = reg_data[np.invert(np.isnan(reg_data.loc[:, "relevance"]))]
long_test = reg_data[np.isnan(reg_data.loc[:, "relevance"])]
sample_size = len(long_train)

long_processed_reg = Regressor(reg_data, sample_size=sample_size)
long_processed_reg.train = long_train[:sample_size]
long_processed_reg.test = long_test[:sample_size]
long_processed_reg.train_test_split()

long_processed_reg.grid_search(model = ridge, auto_params = False)

print(pd.read_csv(f"{sample_size}_ridge_grid_summary.csv", index_col= 0))
coefficients = pd.read_csv(f"{sample_size}_ridge_feature_coefs.csv", index_col= 0)
coefficients["Coefficient"] = coefficients["Coefficient"].abs()
coefficients.sort_values(by = "Coefficient", ascending=False)



constructing search...
fitting...


3128.72s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.72s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.81s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
3128.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed.

      alpha      RMSE
Best     20  0.508086


,Coefficient
spacy_description_stemmed,0.237996
word_in_title_stemmed,0.109902
spacy_title_stem_and_assembled_measurement,0.095073
len_of_query,0.079315
spacy_title_and_measurement_DWH_stem,0.078462
word_in_attributes,0.055600
word_in_description_stemmed,0.041868
spacy_title_and_measurement_LWD_stem,0.028820
spacy_title_and_color_stem,0.009421
spacy_title_and_color_fam_stem,0.002293
